## PEP 634: Structural pattern matching, the switch statement

### The basic match statement

In [1]:
some_variable = 123

In [2]:
match some_variable:
    case 1:
        print('Got 1')
    case 2:
        print('Got 2')
    case _:
        print("Got something else")

Got something else


#### regular if/elif/else ...

In [3]:
if some_variable == 1:
    print("Got 1")
elif some_variable == 2:
    print("Got 2")
else:
    print("Got something else")

Got something else


#### Storing the fallback as a variable

if we name the variable differently,we can store the result:

In [4]:
match some_variable:
    case 1:
        print('Got 1')
    case 2:
        print('Got 2')
    case other:
        print("Got something else")

Got something else


In [5]:
other

123

In [6]:
some_variable == other

True

can we do the equivalent of :
`if some_variable == some_value:`

yes, with a caveat. Since any bare <span style='color:#E75480'>case variable:</span> will result in storing into a variable, we need to have something that does not match that pattern.
The common way to work around this limitation is by introducing a dot:

In [10]:
class Direction:
    LEFT = -1
    RIGHT = 1
    UP = 0
    DOWN = 2

In [8]:
some_direction = Direction.LEFT

In [9]:
match some_direction:
    case Direction.LEFT:
        print("Going left")
    case Direction.RIGHT:
        print("Going right")

Going left


### Matching multiple values in a single case

In [11]:
match some_direction:
    case Direction.LEFT | Direction.LEFT:
        print("Going horizontal")
    case Direction.UP | Direction.DOWN:
        print("Going vertical")

Going horizontal


using the <span style='color:#E75480'>|</span> operator (which is also used for bitwise operations), we can test for mutliple values at the same time.

### Matching values with guards or extra conditions

In [12]:
values = -1, 0, 1

In [14]:
for value in values:
    print("matching", value, end=': ')
    match value:
        case negative if negative < 0:
            print(f"{negative} is smaller than 0.")
        case positive if positive > 0:
            print(f"{positive} is greater than 0.")
        case _:
            print("No match")

matching -1: -1 is smaller than 0.
matching 0: No match
matching 1: 1 is greater than 0.


### Matching lists, tuples, and other sequences

In [16]:
values = (0, 1), (0, 2), (1, 2)

for value in values:
    print("matching", value, end=": ")
    match value:
        case 0, 1:
            print("exactly matched 0, 1")
        case 0, y:
            print(f"matched 0, y with y: {y}")
        case x, y:
            print(f"matched x, y with x, y: {x}, {y}")

matching (0, 1): exactly matched 0, 1
matching (0, 2): matched 0, y with y: 2
matching (1, 2): matched x, y with x, y: 1, 2


### Matching sequence patterns

Let's assume we have a function that takes up to three parameters, <span style='color:#FF90BC'>host, port</span> and <span style='color:#FF90BC'>protocol</span>, we can assume <span style='color:#FF90BC'>443</span> and <span style='color:#FF90BC'>https</span>, respectively, so that only leaves the <span style='color:#FF90BC'>hostname</span> as a required parameter.

In [17]:
def get_uri(*args):
    # set defaults so we only have to store changed variables
    protocol, port, paths = 'https', 443, ()
    match args:
        case(hostname,):
            pass
        case(hostname, port):
            pass
        case(hostname, port, protocol, *paths):
            pass
        case _:
            raise RuntimeError(f"Invalid arguments {args}")
    path = "/".join(paths)
    return f"{protocol}://{hostname}:{port}/{path}"

In [18]:
get_uri('localhost')

'https://localhost:443/'

In [19]:
get_uri('localhost', 12345)

'https://localhost:12345/'

In [20]:
get_uri('localhost', 80, 'http')

'http://localhost:80/'

In [21]:
get_uri('localhost', 80, 'http', 'some', 'paths')

'http://localhost:80/some/paths'

#### capturing sub-patterns

in addition to specifying a variable name to save all values into, we can also store explicit value matches:

In [22]:
values = (0, 1), (0, 2), (1, 2)

In [23]:
for value in values:
    print("[-] Matching", value, end=": ")
    match value:
        case 0 as x, (1 | 2) as y:
            print(f"matched x, y with x, y: {x}, {y}")
        case _:
            print("no match")

[-] Matching (0, 1): matched x, y with x, y: 0, 1
[-] Matching (0, 2): matched x, y with x, y: 0, 2
[-] Matching (1, 2): no match


### Matching dictionaries and other mappings

In [24]:
values = dict(a=0, b=0), dict(a=0, b=1), dict(a=1, b=1)

In [25]:
values

({'a': 0, 'b': 0}, {'a': 0, 'b': 1}, {'a': 1, 'b': 1})

In [26]:
for value in values:
    print("[-] Matching", value, end=": ")
    match value:
        case {'a': 0}:
            print("matched a=0:", value)
        case {'a': 0, 'b': 0}:
            print("matched a=0, b=0:", value)
        case _:
            print("no match")

[-] Matching {'a': 0, 'b': 0}: matched a=0: {'a': 0, 'b': 0}
[-] Matching {'a': 0, 'b': 1}: matched a=0: {'a': 0, 'b': 1}
[-] Matching {'a': 1, 'b': 1}: no match


📌 <span style='color:#F4D03F'>As you can see in the preceding example, matching happens sequentially and it will stop at the first match, not the best match. The second case is never reached in this scenario.</span>

### Matching using isinstance and attributes

In [27]:
class Person:
    def __init__(self, name):
        self.name = name

In [28]:
values = Person('Rick'), Person('Guido')

In [29]:
for value in values:
    match value:
        case Person(name='Rick'):
            print("I found Rick")
        case Person(occupation='Programmer'):
            print("I found a programmer")
        case Person() as person:
            print("I found a person:", person.name)

I found Rick
I found a person: Guido
